In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os

In [2]:
# Reading "gubra_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map.csv", 
                               dtype={"IDPath": "string",
                                      "LabelAbrv": "string",
                                      "raw": "string"})

In [3]:
#Creating the 'LabelID'
brain_gubra_map['LabelID'] = brain_gubra_map['IDPath'].str.split('/').str[-2]

In [4]:
#Creating the right gruba intensities template
right_gubra = brain_gubra_map[['IDPath','LabelID','raw','LabelAbrv']]
right_gubra[['LabelID']] = right_gubra[['LabelID']].astype(int)
right_gubra['raw'] = right_gubra['raw'].str.replace(',','',regex=True)
right_gubra['raw'] = right_gubra['raw'].str.replace(' ','_',regex=True)
right_gubra['raw'] = 'right_' + right_gubra['raw']
right_gubra['LabelAbrv'] = 'R' + right_gubra['LabelAbrv']
right_gubra

C:\Users\intri\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\intri\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\intri\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,IDPath,LabelID,raw,LabelAbrv
0,/997/8/567/623/477/278/23/,23,right_Anterior_amygdalar_area,RAAA
1,/997/8/567/688/695/315/31/39/935/,935,right_Anterior_cingulate_area_dorsal_part_layer_1,RACAd1
2,/997/8/567/688/695/315/31/39/211/,211,right_Anterior_cingulate_area_dorsal_part_laye...,RACAd2/3
3,/997/8/567/688/695/315/31/39/1015/,1015,right_Anterior_cingulate_area_dorsal_part_layer_5,RACAd5
4,/997/8/567/688/695/315/31/39/919/,919,right_Anterior_cingulate_area_dorsal_part_laye...,RACAd6a
...,...,...,...,...
588,/997/1009/967/933/413/,413,right_vestibular_nerve,RvVIIIn
589,/997/8/343/1065/354/370/765/,765,right_Nucleus_x,Rx
590,/997/8/343/1065/354/370/773/,773,right_Hypoglossal_nucleus,RXII
591,/997/8/343/1065/354/370/781/,781,right_Nucleus_y,Ry


In [5]:
#Creating the left gruba intensities template
left_gubra = brain_gubra_map[['IDPath','LabelID','raw','LabelAbrv']]
left_gubra[['LabelID']] = left_gubra[['LabelID']].astype(int)
left_gubra['raw'] = left_gubra['raw'].str.replace(',','',regex=True)
left_gubra['raw'] = left_gubra['raw'].str.replace(' ','_',regex=True)
left_gubra['raw'] = 'left_' + left_gubra['raw']
left_gubra = left_gubra[left_gubra.LabelID != 1]
left_gubra['LabelID'] = left_gubra['LabelID'] + 20000
left_gubra['LabelAbrv'] = 'L' + left_gubra['LabelAbrv']
left_gubra

C:\Users\intri\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\intri\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\intri\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,IDPath,LabelID,raw,LabelAbrv
0,/997/8/567/623/477/278/23/,20023,left_Anterior_amygdalar_area,LAAA
1,/997/8/567/688/695/315/31/39/935/,20935,left_Anterior_cingulate_area_dorsal_part_layer_1,LACAd1
2,/997/8/567/688/695/315/31/39/211/,20211,left_Anterior_cingulate_area_dorsal_part_layer...,LACAd2/3
3,/997/8/567/688/695/315/31/39/1015/,21015,left_Anterior_cingulate_area_dorsal_part_layer_5,LACAd5
4,/997/8/567/688/695/315/31/39/919/,20919,left_Anterior_cingulate_area_dorsal_part_layer_6a,LACAd6a
...,...,...,...,...
588,/997/1009/967/933/413/,20413,left_vestibular_nerve,LvVIIIn
589,/997/8/343/1065/354/370/765/,20765,left_Nucleus_x,Lx
590,/997/8/343/1065/354/370/773/,20773,left_Hypoglossal_nucleus,LXII
591,/997/8/343/1065/354/370/781/,20781,left_Nucleus_y,Ly


In [6]:
gubra_intensities_template = right_gubra.append(left_gubra)

gubra_intensities_template

,IDPath,LabelID,raw,LabelAbrv
0,/997/8/567/623/477/278/23/,23,right_Anterior_amygdalar_area,RAAA
1,/997/8/567/688/695/315/31/39/935/,935,right_Anterior_cingulate_area_dorsal_part_layer_1,RACAd1
2,/997/8/567/688/695/315/31/39/211/,211,right_Anterior_cingulate_area_dorsal_part_laye...,RACAd2/3
3,/997/8/567/688/695/315/31/39/1015/,1015,right_Anterior_cingulate_area_dorsal_part_layer_5,RACAd5
4,/997/8/567/688/695/315/31/39/919/,919,right_Anterior_cingulate_area_dorsal_part_laye...,RACAd6a
...,...,...,...,...
588,/997/1009/967/933/413/,20413,left_vestibular_nerve,LvVIIIn
589,/997/8/343/1065/354/370/765/,20765,left_Nucleus_x,Lx
590,/997/8/343/1065/354/370/773/,20773,left_Hypoglossal_nucleus,LXII
591,/997/8/343/1065/354/370/781/,20781,left_Nucleus_y,Ly


In [7]:
gubra_intensities_template.to_csv('gubra_intensities_template.csv', index=False)